### Setup langfuse tracing

In [1]:
from my_utils import setup_langfuse_tracer
from my_config import MyConfig

langfuse_handler = setup_langfuse_tracer()
my_config = MyConfig()

In [2]:
# pip install -r requirements.txt

### Create a model from the Ollama model that is running on runpod

In [3]:
# from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama
remote_url = f"https://{my_config.OLLAMA_INFERENCE_RUNPOD_ID}-11434.proxy.runpod.net"
model = ChatOllama(
    base_url=remote_url
    # , model='deepseek-r1:32b'
    , model='llama4:scout'
)


In [4]:
response = model.invoke("What is the capitalof India?", config={"callbacks": [langfuse_handler]})
response.content

'The capital of India is New Delhi.'

### LangGraph imports

In [5]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
# from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

### Define Our State

##### Let’s define what information Alfred needs to track during the email processing workflow:

In [6]:
class EmailState(TypedDict):
    # The email being processed
    email: Dict[str, Any]  # Contains subject, sender, body, etc.

    # Category of the email (inquiry, complaint, etc.)
    email_category: Optional[str]

    # Email Classification Description
    classification_description: Optional[str]

    # Email Classification Description
    analyze_classification_response: Optional[str]

    # Email Classification Result
    spam_result: Optional[str]

    # Reason why the email was marked as spam
    spam_reason: Optional[str]

    # Analysis and decisions
    is_spam: Optional[bool]
    
    # Response generation
    email_draft: Optional[str]
    
    # Processing metadata
    messages: List[Dict[str, Any]]  # Track conversation with LLM for analysis

#### Define Our Nodes
Now, let’s create the processing functions that will form our nodes:

In [7]:
def read_email(state: EmailState):
    """Alfred reads and logs the incoming email"""
    email = state["email"]
    
    # Here we might do some initial preprocessing
    print(f"Alfred is processing an email from {email['sender']} with subject: {email['subject']}")
    
    # No state changes needed here
    return {}

def classify_email(state: EmailState):
    """Alfred uses an LLM to determine if the email is spam or legitimate"""
    email = state["email"]
    
    # Prepare our prompt for the LLM
    prompt = f"""
    As Alfred the butler, analyze this email and try to reason about if it is spam or legitimate.
    Please provide the classification conclusion and a brief explanation within **classification_conclusion** block.
    If it is spam, provide a reason within **spam_reason** block.
    
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}
    
    First, determine if this email is spam. If it is spam, explain why.
    If it is legitimate, categorize it (inquiry, complaint, thank you, etc.).
    """
    
    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    
    # Simple logic to parse the response (in a real app, you'd want more robust parsing)
    response_text = response.content.lower()
    
    # Return state updates
    return {
        "classification_description": response_text.lower().split("**classification_conclusion**")[-1].strip().split('**')[0] if "**classification_conclusion**" in response_text else response_text,
        "spam_reason": response_text.lower().split("**spam_reason**")[-1].strip().split('**')[0] if "**spam_reason**" in response_text else response_text,
        "messages": state.get("messages", []) + [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response.content}
        ]
    }


def analyze_classification(state: EmailState):
    """Alfred analyzes the classification result and updates state accordingly"""
    classification_output = state["classification_description"]

    # Prepare our prompt for the LLM
    prompt = f"""
    As Alfred the butler, figure out if the mail is spam or not based on the classification description.
    Use the **classification_conclusion** block from the input.
    After analysing the output state `spam` or `not spam` in clear CAPITAL letters. Provide this output in **final_classification_result** block

    
    This is the classification result in detail:
        **classification_conclusion**
        {classification_output}
    """
    
    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    
    response_text = response.content
    # Simple logic to parse the response (in a real app, you'd want more robust parsing)
    spam_result = response_text.split("**final_classification_result**")[-1].strip() if "**final_classification_result**" in response_text else response_text
    is_spam = "SPAM" in spam_result and "NOT SPAM" not in spam_result
    spam_reason = state.get("spam_reason", "No reason provided")
    
    # Determine category if legitimate
    email_category = None
    if not is_spam:
        categories = ["inquiry", "complaint", "thank you", "request", "information"]
        for category in categories:
            if category in response_text:
                email_category = category
                break
    
    # Update messages for tracking
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]
    
    # Return state updates
    return {
        "is_spam": is_spam,
        "spam_reason": spam_reason,
        "email_category": email_category,
        "messages": new_messages,
        "analyze_classification_response": response_text,
        "spam_result": spam_result
    }

def handle_spam(state: EmailState):
    """Alfred discards spam email with a note"""
    print(f"Alfred has marked the email as spam. Reason: {state['spam_reason']}")
    print("The email has been moved to the spam folder.")
    print(f"Email is marked as spam with reason: {state['spam_reason']}")

    # We're done processing this email
    return {}

def draft_response(state: EmailState):
    """Alfred drafts a preliminary response for legitimate emails"""
    email = state["email"]
    category = state["email_category"] or "general"
    
    # Prepare our prompt for the LLM
    prompt = f"""
    As Alfred the butler, draft a polite preliminary response to this email.
    
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}
    
    This email has been categorized as: {category}
    
    Draft a brief, professional response that Mr. Hugg can review and personalize before sending.
    """
    
    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    
    # Update messages for tracking
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]
    
    # Return state updates
    return {
        "email_draft": response.content,
        "messages": new_messages
    }

def notify_mr_hugg(state: EmailState):
    """Alfred notifies Mr. Hugg about the email and presents the draft response"""
    email = state["email"]
    
    print("\n" + "="*50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print(f"Category: {state['email_category']}")
    print("\nI've prepared a draft response for your review:")
    print("-"*50)
    print(state["email_draft"])
    print("="*50 + "\n")
    
    # We're done processing this email
    return {}

#### Define Our Routing Logic
We need a function to determine which path to take after classification:

In [8]:
def route_email(state: EmailState) -> str:
    """Determine the next step based on spam classification"""
    if state["is_spam"]:
        return "spam"
    else:
        return "legitimate"

#### Create the StateGraph and Define Edges
Now we connect everything together:

In [9]:
# Create the graph
email_graph = StateGraph(EmailState)

# Add nodes
email_graph.add_node("read_email", read_email)
email_graph.add_node("classify_email", classify_email)
email_graph.add_node("analyze_classification", analyze_classification)
email_graph.add_node("handle_spam", handle_spam)
email_graph.add_node("draft_response", draft_response)
email_graph.add_node("notify_mr_hugg", notify_mr_hugg)

# Start the edges
email_graph.add_edge(START, "read_email")
# Add edges - defining the flow
email_graph.add_edge("read_email", "classify_email")
email_graph.add_edge("classify_email", "analyze_classification")

# Add conditional branching from classify_email
email_graph.add_conditional_edges(
    "analyze_classification",
    route_email,
    {
        "spam": "handle_spam",
        "legitimate": "draft_response"
    }
)

# Add the final edges
email_graph.add_edge("handle_spam", END)
email_graph.add_edge("draft_response", "notify_mr_hugg")
email_graph.add_edge("notify_mr_hugg", END)

# Compile the graph
compiled_graph = email_graph.compile()

#### Run the Application
Let’s test our graph with a legitimate email and a spam email:

In [10]:
# Example legitimate email
legitimate_email = {
    "sender": "john.smith@example.com",
    "subject": "Question about your services",
    "body": "Dear Mr. Hugg, I was referred to you by a colleague and I'm interested in learning more about your consulting services. Could we schedule a call next week? Best regards, John Smith"
}

# Example spam email
spam_email = {
    "sender": "winner@lottery-intl.com",
    "subject": "YOU HAVE WON $5,000,000!!!",
    "body": "CONGRATULATIONS! You have been selected as the winner of our international lottery! To claim your $5,000,000 prize, please send us your bank details and a processing fee of $100."
}

# Process the legitimate email
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke({
    "email": legitimate_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
}, config={"callbacks": [langfuse_handler]})

# Process the spam email
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke({
    "email": spam_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
}, config={"callbacks": [langfuse_handler]})


Processing legitimate email...
Alfred is processing an email from john.smith@example.com with subject: Question about your services

Sir, you've received an email from john.smith@example.com.
Subject: Question about your services
Category: None

I've prepared a draft response for your review:
--------------------------------------------------
Here's a draft response:

Dear Mr. Smith,

Thank you for reaching out and for the referral from your colleague. I'm pleased to hear that you're interested in learning more about my consulting services.

I'd be delighted to schedule a call with you next week. Could you please let me know a few dates and times that work for you, and I'll do my best to accommodate them?

I look forward to speaking with you soon.

Best regards,
Mr. Hugg


Processing spam email...
Alfred is processing an email from winner@lottery-intl.com with subject: YOU HAVE WON $5,000,000!!!
Alfred has marked the email as spam. Reason: : the email exhibits several red flags common